In [10]:
import Erdos_Renyi as er 
from pyhawkes.models import DiscreteTimeNetworkHawkesModelSpikeAndSlab
from pyhawkes.internals import parents
import json 
import pandas as pd 
from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Parameters
p = 0.5
dt_max=200
subset = 200
K = subset

In [3]:
# Real data
settings = json.load(open("settings.json","r"))
# Path to file
events_embd = settings["out_path"]+"concepts_date_embd.csv"
# load events
print("Reading in data")
events = pd.read_csv(events_embd,header=None)


Reading in data


In [4]:
L = events.head(subset).iloc[:,:-1].values
S_real = np.eye(subset,dtype=np.int)#events.head(subset)[300].values

In [11]:
ld_network = er.LatentDistanceAdjacencyModel(K=K, L = L, dim=2, v=None, alpha=1.0, beta=1.0,kappa=1.0,p = p)

weight_model = er.SpikeAndSlabGammaWeights(model = ld_network, parallel_resampling=False)

> c:\users\miha\dropbox\institut\phd\pointprocess\code\eventgraphmodelling\erdos_renyi.py(719)resample()
-> # Resample W | A
(Pdb) n
> c:\users\miha\dropbox\institut\phd\pointprocess\code\eventgraphmodelling\erdos_renyi.py(722)resample()
-> # Resample A given W
(Pdb) n
> c:\users\miha\dropbox\institut\phd\pointprocess\code\eventgraphmodelling\erdos_renyi.py(725)resample()
-> else:
(Pdb) 
--Return--
> c:\users\miha\dropbox\institut\phd\pointprocess\code\eventgraphmodelling\erdos_renyi.py(725)resample()->None
-> else:
(Pdb) n
--Return--
> c:\users\miha\dropbox\institut\phd\pointprocess\code\eventgraphmodelling\erdos_renyi.py(586)__init__()->None
-> self.resample()
(Pdb) n
--Return--
> <ipython-input-11-3e49d05f168b>(3)<module>()->None
-> weight_model = er.SpikeAndSlabGammaWeights(model = ld_network, parallel_resampling=False)
(Pdb) n
> c:\users\miha\anaconda2\envs\python3\lib\site-packages\ipython\core\interactiveshell.py(2865)run_code()
-> sys.excepthook = old_excepthook
(Pdb) 
> c:\use

In [12]:
class empty_model():
    def __init__(self,S):
        self.dt = 0
        self.K = S.shape[0]
        self.B = 0
        self.bias_model = None

In [ ]:
data  = parents.DiscreteTimeParents(model = None,S = S_real, T = S_real.shape[0],F=None)
weight_model.resample(data)
#A,W = weight_model.A,weight_model.W
# resample graph with new data 
ld_network.resample(data=[weight_model.A,weight_model.W])



In [ ]:
G = nx.from_numpy_matrix(self.A)
nx.draw(G,with_labels=True,with_edges=True)